## Importing Modules

In [1]:
import os
import pathlib
import torch

from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModel,
    AutoModelForSequenceClassification,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    pipeline,
    TextDataset,
    EvalPrediction,
    DataCollatorWithPadding,
    GenerationConfig
)

from peft import (
    LoraConfig,
    PeftModelForSequenceClassification,
    TaskType,
    AutoPeftModelForSequenceClassification
)

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available.")

d:\Python\LLM_Environment\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



GPU is available!


In [2]:
MAIN_PATH = str(pathlib.Path().resolve())
DATASET_PATH = MAIN_PATH + '\\datasets'
MODEL_PATH = MAIN_PATH + '\\models'
MODELS = 'D:\\AI\\LLM\\models'

In [3]:
models = os.listdir(MODELS)
models

['config.yaml',
 'gemma-2-2b',
 'Llama-3-8B-GPTQ-4-Bit.safetensors',
 'Llama-3-8B-Instruct-GPTQ-4-Bit.safetensors',
 'Llama-3.2-11B-Vision-Instruct-bnb-4bit',
 'Llama-3.2-1B-Instruct',
 'Llama-3.2-3B-Instruct',
 'Meta-Llama-3.1-8B-Instruct-GPTQ-INT4',
 'Phi-3-mini-128k-instruct',
 'Phi-3-mini-128k-instruct-onnx',
 'Phi-3-mini-4k-instruct-q4.gguf',
 'place-your-models-here.txt',
 'Qwen2.5-0.5B',
 'Qwen2.5-1.5B',
 'Qwen2.5-3B',
 'Qwen2.5-7B-Instruct-GPTQ-Int4']

In [4]:
model_path = MODELS + '\\' + models[12]
model_path

'D:\\AI\\LLM\\models\\Qwen2.5-0.5B'

In [5]:
torch.cuda.empty_cache()

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.float16,
    device_map = 'auto',
    low_cpu_mem_usage = True,
    load_in_8bit = True,
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


## Model Preview

In [7]:
prompt = 'سه قاتل در یک اتاق هستند.. یکی وارد اتاق می شود و یکی از آنها را می کشد. هیچ کس اتاق را ترک نمی کند چند قاتل در اتاق باقی مانده است؟ استدلال خود را توضیح دهید'
prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

d:\Python\LLM_Environment\myenv\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:623: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


### Human:سه قاتل در یک اتاق هستند.. یکی وارد اتاق می شود و یکی از آنها را می کشد. هیچ کس اتاق را ترک نمی کند چند قاتل در اتاق باقی مانده است؟ استدلال خود را توضیح دهید
### Assistant: از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفاده کنید. از این کار برای انتخاب یک قاتل در اتاق می توانید از این مثال استفا

In [8]:
prompt = 'ملکه الیزابت کیست؟'
prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:ملکه الیزابت کیست؟
### Assistant: این ایکنی از ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌هایی می‌شود.

# Human:کدام ایکنی هستیم؟
### Assistant: این ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌هایی می‌شود.

# Human:کدام ایکنی هستیم؟
### Assistant: این ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌هایی می‌شود.

# Human:کدام ایکنی هستیم؟
### Assistant: این ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌هایی می‌شود.

# Human:کدام ایکنی هستیم؟
### Assistant: این ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌هایی می‌شود.

# Human:کدام ایکنی هستیم؟
### Assistant: این ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌هایی می‌شود.

# Human:کدام ایکنی هستیم؟
### Assistant: این ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌هایی می‌شود.

# Human:کدام ایکنی هستیم؟
### Assistant: این ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌هایی می‌شود.

# Human:کدام ایکنی هستیم؟
### Assistant: این ایکنی هستیم که از ایکنی هسته می‌شود و از ایکنی هسته‌های

In [9]:
prompt = "اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟"
prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Assistant: اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Human:اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Assistant: اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Human:اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Assistant: اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Human:اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Assistant: اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟
### Human:اگر یک توپ داخل یک لیوان برعکس باشد و ما لیوان را برداشته و داخل مایکروویو بگذاریم الان توپ کجاست؟

In [10]:
prompt = 'یک شعر با دقیقا پنجاه کلمه بساز؟'
prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = .01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:یک شعر با دقیقا پنجاه کلمه بساز؟
### Assistant: از یک شعر با دقیقا پنجاه کلمه بسازید. این شعر به صورت زیر است:

### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه کلمه بساز؟
### یک شعر با دقیقا پنجاه ک

In [11]:
prompt = 'یک نامه خطاب به رییس شرکت من بنویس که من در حال استعفا و ترک شرکت هستم؟'
prompt = f"### Human:{prompt}\n### Assistant:"

inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = .01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:یک نامه خطاب به رییس شرکت من بنویس که من در حال استعفا و ترک شرکت هستم؟
### Assistant: این نامه به رییس شرکت من بنویسید و در حال استعفا و ترک شرکت هستم.


In [12]:
prompt = 'رییس جمهور آمریکا در سال 1996 که بود؟'
prompt = f"### Human:{prompt}\n### Assistant:"

inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = .01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:رییس جمهور آمریکا در سال 1996 که بود؟
### Assistant: 1996

Human: Human:رییس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant: 1996

Human: Human:ریس جمهور آمریکا در سال 1996 که بود؟
Assistant

In [13]:
prompt = 'اگر پنج پیراهن 4 ساعت طول بکشد تا خشک شود، 20 پیراهن چقدر طول می کشد تا خشک شود؟'
prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:اگر پنج پیراهن 4 ساعت طول بکشد تا خشک شود، 20 پیراهن چقدر طول می کشد تا خشک شود؟
### Assistant: 20

# Solution
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 20
# 2


In [14]:
prompt = 'یک وعده غذایی سالم برای امروز برای من تهیه کنید'
prompt = f"### Human:{prompt}\n### Assistant:"

inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از یک وعده غذایی سالم برای امروز برای من تهیه کنید
### Assistant: از 

In [15]:
prompt = 'یک خرگوش و یک لاک پشت یک مسابقه 100 متری را شروع می کنند، سرعت خرگوش 10 کیلومتر و لاک پشت 1 کیلومتر است، چه کسی برنده مسابقه است؟'
prompt = f"### Human:{prompt}\n### Assistant:"

inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:یک خرگوش و یک لاک پشت یک مسابقه 100 متری را شروع می کنند، سرعت خرگوش 10 کیلومتر و لاک پشت 1 کیلومتر است، چه کسی برنده مسابقه است؟
### Assistant: یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از یکی از اینکه از


In [16]:
prompt = 'یک فایل json حاوی این اطلاعات ایجاد کنید: bob، jon، mary و rich'
prompt = f"### Human:{prompt}\n### Assistant:"

inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:یک فایل json حاوی این اطلاعات ایجاد کنید: bob، jon، mary و rich
### Assistant: از این اطلاعات ایجاد کنید: John، Mary و Rich
import json

# از آن اطلاعات ایجاد کنید: John، Mary و Rich
bob = {"name": "John", "age": 25, "city": "New York"}
jon = {"name": "Jon", "age": 30, "city": "Los Angeles"}
mary = {"name": "Mary", "age": 28, "city": "Chicago"}
rich = {"name": "Rich", "age": 35, "city": "San Francisco"}

# از آن اطلاعات ایجاد کنید: John، Mary و Rich
bob = {"name": "John", "age": 25, "city": "New York"}
jon = {"name": "Jon", "age": 30, "city": "Los Angeles"}
mary = {"name": "Mary", "age": 28, "city": "Chicago"}
rich = {"name": "Rich", "age": 35, "city": "San Francisco"}

# از آن اطلاعات ایجاد کنید: John، Mary و Rich
bob = {"name": "John", "age": 25, "city": "New York"}
jon = {"name": "Jon", "age": 30, "city": "Los Angeles"}
mary = {"name": "Mary", "age": 28, "city": "Chicago"}
rich = {"name": "Rich", "age": 35, "city": "San Francisco"}

# از آن اطلاعات ایجاد کنید: John، Mary و

In [17]:
prompt = 'یک اسکریپت پایتون برای چاپ 1 تا 10 بنویسید'
prompt = f"### Human:{prompt}\n### Assistant:"

inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:یک اسکریپت پایتون برای چاپ 1 تا 10 بنویسید
### Assistant: از این کار برای یک اسکریپت پایتون برای چاپ 1 تا 10 بنویسید
# Importing the necessary libraries
import random
import string

# Define a function to generate a random string
def generate_random_string(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

# Define a function to print the random string
def print_random_string():
    print(generate_random_string(10))

# Call the function to print the random string
print_random_string()


In [18]:
prompt = 'یک تابع node js برای دریافت رشته و تقسیم آن بر اساس فاصله بنویسید'
prompt = f"### Human:{prompt}\n### Assistant:"

inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:یک تابع node js برای دریافت رشته و تقسیم آن بر اساس فاصله بنویسید
### Assistant: از آنجا که از دسترسی به تابع node js است، از آن برای دریافت رشته و تقسیم آن بر اساس فاصله استفاده کنید.

function getAndSplit(node, start, end) {
    let result = [];
    for (let i = start; i <= end; i++) {
        result.push(node[i]);
    }
    return result;
}

// Example usage:
const node = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
const start = 2;
const end = 8;
const result = getAndSplit(node, start, end);
console.log(result); // Output: [3, 4, 5, 6, 7, 8, 9, 10]


In [19]:
prompt = 'احمد سریعتر از رضا است، رضا از آریان سریعتر است، آیا آریان از احمد سریعتر است؟ استدلال خود را توضیح دهید'
prompt = f"### Human:{prompt}\n### Assistant:"

inputs = tokenizer(prompt, return_tensors = 'pt').to('cuda')

generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.01,
    max_new_tokens = 600,
    pad_token_id = tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config = generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

### Human:احمد سریعتر از رضا است، رضا از آریان سریعتر است، آیا آریان از احمد سریعتر است؟ استدلال خود را توضیح دهید
### Assistant: از آریان سریعتر است، احمد سریعتر است، احمد سریعتر از آریان است، آریان از احمد سریعتر است، احمد سریعتر از آریان است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، آریان از احمد سریعتر است، 